In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim 
import torch.utils.data as Data 
import torch.nn.functional as F
dtype = torch.FloatTensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/home/zhenglicheng/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
print(torch.__version__)

1.13.1+cu117


In [3]:
# write a class to read data.txt with two attributes

import jieba
import json 

def load_data(file_path):
    dataset = []
    for line in open(file_path, "r"):
        instance = json.loads(line)
        sent = [x for x in jieba.cut(instance['sentence'])]
        label = instance['label']
        dataset.append((sent, label))
    return dataset

test_dataset= load_data("./tnews_public/test.json")
train_dataset = load_data("./tnews_public/train.json")
print(len(test_dataset))
print(test_dataset[:3])

print(len(train_dataset))
print(train_dataset[:3])
    


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.440 seconds.
Prefix dict has been built successfully.


10000
[(['江', '疏影', '甜甜', '圈自', '拍', '，', '迷', '之', '角度', '竟', '这么', '好看', '，', '美', '吸引', '一切', '事物'], '102'), (['以色列', '大规模', '空袭', '开始', '！', '伊朗', '多个', '军事', '目标', '遭遇', '打击', '，', '誓言', '对', '等', '反击'], '110'), (['出栏', '一头', '猪', '亏损', '300', '元', '，', '究竟', '谁', '能', '笑', '到', '最后', '！'], '104')]
53360
[(['上课时', '学生', '手机', '响个', '不停', '，', '老师', '一怒之下', '把', '手机', '摔', '了', '，', '家长', '拿', '发票', '让', '老师', '赔', '，', '大家', '怎么', '看待', '这种', '事', '？'], '108'), (['商赢', '环球', '股份', '有限公司', '关于', '延期', '回复', '上海证券交易所', '对', '公司', '2017', '年', '年度报告', '的', '事后', '审核', '问询', '函', '的', '公告'], '104'), (['通过', '中介', '公司', '买', '了', '二手房', '，', '首付', '都', '付', '了', '，', '现在', '卖家', '不想', '卖', '了', '。', '怎么', '处理', '？'], '106')]


In [4]:

dataset = train_dataset +  test_dataset
from collections import Counter
def make_data(dataset):
    counter = Counter()  
    for (sent, label) in dataset:
        counter.update(sent)
    # sorted(iterable, cmp=None, key=None, reverse=False)
    vocab = [(word, count) for (word, count) in counter.items() if count > 1]
    sorted_vocab = sorted(vocab, key=lambda x: x[1], reverse=False)
    word2idx ={'unk':0}
    word2idx.update({word: i+1 for i, (word, count) in enumerate(sorted_vocab)})
   
    idx2word = {i: word for (word, i) in word2idx.items()}
    
    return word2idx, idx2word
        # inputs.appendword2idx[n]([ for n in sen.split()])
word2idx, idx2word = make_data(dataset)
# targets = []
# for out in labels:
# targets.append(out) return inputs, targets
print(word2idx)
print(idx2word)
vocab_size =len(word2idx)


{'unk': 0, '得花': 1, '剃须刀': 2, '办展': 3, '央地': 4, '以谣': 5, '山梨': 6, '酯': 7, '高烧': 8, '吓瘫': 9, '千家万户': 10, '银元': 11, '完会': 12, 'CS35': 13, '大猩猩': 14, '绚丽': 15, '上百人': 16, '焕驰': 17, '咣当': 18, '平昌': 19, '主创': 20, '张鹤伦': 21, '水枪': 22, '矿泉水': 23, '6.9': 24, 'ISIS': 25, '华盛顿': 26, '独播': 27, '偷走': 28, '美墅': 29, '郑板桥': 30, '勾': 31, '旱魃': 32, '臣': 33, '动真格': 34, '科博会': 35, '青白江': 36, '不加': 37, '何鸿燊': 38, 'M82A1': 39, '飞地': 40, '饭堂': 41, '原图': 42, '拍摄者': 43, '生态化': 44, '煤气罐': 45, '几千吨': 46, '赖账': 47, 'DPF': 48, '多半': 49, '经络': 50, '不通': 51, '励耘': 52, '餐饮行业': 53, '隐含': 54, '遥控器': 55, '太强大': 56, 'Top5': 57, 'GMIC': 58, '潮头': 59, '村姑': 60, '掌管': 61, '4.7': 62, '一扇': 63, '起跑': 64, '火影': 65, 'Z5': 66, '学生会': 67, '附属': 68, '住进': 69, '晨起': 70, '求问': 71, '发紫': 72, '嘟': 73, '一响': 74, '三无': 75, '流通股东': 76, '助兴': 77, '绘出': 78, '纽约州': 79, '一万五': 80, '华强': 81, '月刊': 82, '抢鲜': 83, '刚才': 84, '牛首山': 85, '武侠剧': 86, '神箭': 87, '潘迎紫': 88, '美若': 89, '催肥': 90, '肉羊': 91, '增重': 92, '水友': 93, '核计划': 94, '止损': 95, '技术指标': 

In [5]:
import torch 
import torch.optim as optim 
import torch.utils.data as Data 
import torch.nn.functional as F

def padding_and_cut(idx_seq, max_length):
    # padding 补零
    if len(idx_seq) < max_length:
        idx_seq += [0]*(max_length-len(idx_seq))
    else:
        # 截取
        idx_seq = idx_seq[:max_length]
    return idx_seq

def serialization(dataset, word2idx, max_length=20):
    input_ids = []
    label_ids = []
    for sent, label in dataset:
        idx_seq = []
        for word in sent:
            if word in word2idx:
                idx = word2idx[word]
            else:
                idx = word2idx["unk"]
            idx_seq.append(idx)
        input_ids.append(padding_and_cut(idx_seq, max_length))
        label_ids.append(int(label)-100)
    return input_ids, label_ids

batch_size = 4
input_ids, label_ids=  serialization(test_dataset, word2idx)
print(input_ids)
print(label_ids)
inputs = torch.LongTensor(input_ids)
labels = torch.LongTensor(label_ids)
dataset = Data.TensorDataset(inputs,labels) 
data_loader = Data.DataLoader(dataset, batch_size, True)

[[30199, 27185, 12137, 0, 34885, 35460, 26370, 35297, 28265, 34770, 35332, 34737, 35460, 35290, 33838, 33840, 24473, 0, 0, 0], [35309, 33754, 33882, 35203, 35457, 35317, 32448, 34659, 34131, 34167, 34139, 35460, 28976, 35431, 35277, 33741, 0, 0, 0, 0], [19516, 30964, 33797, 34203, 34690, 35244, 35460, 35040, 35405, 35425, 34958, 35403, 35228, 35457, 0, 0, 0, 0, 0, 0], [33683, 28153, 35458, 29397, 35378, 35382, 0, 35459, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [34829, 33337, 34732, 22047, 35460, 35450, 34609, 35343, 13104, 35412, 34622, 35384, 26428, 35458, 34437, 35459, 0, 0, 0, 0], [34768, 10395, 35458, 35407, 35455, 19413, 12509, 35300, 35458, 31256, 28378, 0, 0, 0, 0, 0, 0, 0, 0, 0], [27535, 35456, 34890, 28105, 35455, 35398, 34689, 35455, 0, 35455, 34549, 35033, 35266, 30526, 33942, 35326, 0, 0, 0, 0], [35261, 35270, 35288, 27287, 35460, 35425, 34763, 35415, 35233, 26293, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [32601, 32269, 35460, 25247, 35389, 0, 35459, 34726, 28724, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
import torch.nn as nn

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_classes):
        super(TextCNN, self).__init__()
        self.W = nn.Embedding(vocab_size, embedding_size)
        output_channel = 20
        self.conv = nn.Sequential(
            nn.Conv2d(1, output_channel, (2, embedding_size)),
            nn.ReLU(),
            nn.MaxPool2d((output_channel-1, 1)))
        self.fc = nn.Linear(output_channel, num_classes)
    def forward(self, X): 
        '''
        X: [batch_size, sequence_length]
        '''
        batch_size = X.shape[0]
        # [batch_size, sequence_length, embedding_size] 
        embedding_X = self.W(X)
        embedding_X = embedding_X.unsqueeze(1)  
        conved = self.conv(embedding_X)  
        flatten = conved.view(batch_size, -1)
        #print(flatten.size())
        output = self.fc(flatten)
        
        return output


In [7]:
embedding_size = 20
num_classes = 17
batch_size = 4
from tqdm import tqdm
model = TextCNN(vocab_size, embedding_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss().to(device) 
optimizer = optim.Adam(model.parameters(), lr=1e-2)
# Training
for epoch in tqdm(range(500)):
    for batch_x, batch_y in data_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device) 
        pred = model(batch_x)
        loss = criterion(pred, batch_y)
        if (epoch + 1) %10 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step()


  1%|          | 4/500 [00:30<1:02:00,  7.50s/it]


KeyboardInterrupt: 